In [2]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from keras.utils import np_utils
from sklearn.datasets import load_files
from keras.preprocessing import image


In [33]:
train_files=load_files('train/')
train_imgpaths=train_files['filenames']
train_targets=np_utils.to_categorical(train_files['target'],len(train_files['target_names']))
target_names=dict(enumerate(train_files.target_names))

In [4]:
def path_to_tensor(path):
    #load image
    img=image.load_img(path,target_size=[224,224])
    imgarray=image.img_to_array(img)
    imgarray=np.expand_dims(imgarray,axis=0)
    return imgarray

In [5]:
def paths_to_tensor(paths):
    listoftensors=[path_to_tensor(path) for path in paths]
    return np.vstack(listoftensors)

In [6]:
train_tensors=paths_to_tensor(train_imgpaths)


In [7]:
train_tensors=train_tensors/float(255)
train_tensors,valid_tensors,test_tensors=train_tensors[:-1700],train_tensors[-1700:-950],train_tensors[-950:]
train_targets,valid_targets,test_targets=train_targets[:-1700],train_targets[-1700:-950],train_targets[-950:]

In [8]:
train_gen=image.ImageDataGenerator(horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1)
train_gen.fit(train_tensors)

In [9]:
from keras.layers import Dense,Convolution2D,MaxPooling2D,Dropout,Flatten
from keras.models import Sequential

In [10]:
snn=Sequential()
snn.add(Convolution2D(kernel_size=[3,3],filters=16,activation='relu',input_shape=[224,224,3]))
snn.add(MaxPooling2D(pool_size=[3,3]))
snn.add(Convolution2D(kernel_size=[3,3],filters=32,activation='relu'))
snn.add(MaxPooling2D(pool_size=[3,3]))
snn.add(Convolution2D(kernel_size=[3,3],filters=64,activation='relu'))
snn.add(MaxPooling2D(pool_size=[3,3]))
snn.add(Convolution2D(kernel_size=[3,3],filters=128,activation='relu'))
snn.add(MaxPooling2D(pool_size=[3,3]))
snn.add(Flatten())
snn.add(Dense(units=10,activation='relu'))
snn.add(Dropout(0.2))
snn.add(Dense(units=12,activation='softmax'))
snn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 22, 22, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 5, 5, 128)         73856     
__________

In [11]:
snn.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='rmsprop')

In [15]:
epochs=100
batch_size=30

from keras.callbacks import ModelCheckpoint

checkpointer=ModelCheckpoint(filepath='snnweights_augmentation.best.hdf5',save_best_only=True,verbose=1)

snn.fit_generator(train_gen.flow(train_tensors,train_targets,batch_size=batch_size),
                  steps_per_epoch=train_tensors.shape[0]//batch_size,
                  validation_data=[valid_tensors,valid_targets],
                    epochs=epochs,verbose=2,callbacks=[checkpointer])

Epoch 1/100
Epoch 00000: val_loss improved from inf to 2.41965, saving model to snnweights_augmentation.best.hdf5
21s - loss: 2.4700 - acc: 0.1149 - val_loss: 2.4197 - val_acc: 0.2080
Epoch 2/100
Epoch 00001: val_loss improved from 2.41965 to 2.16488, saving model to snnweights_augmentation.best.hdf5
19s - loss: 2.3092 - acc: 0.1980 - val_loss: 2.1649 - val_acc: 0.2920
Epoch 3/100
Epoch 00002: val_loss improved from 2.16488 to 1.94039, saving model to snnweights_augmentation.best.hdf5
19s - loss: 2.1385 - acc: 0.2823 - val_loss: 1.9404 - val_acc: 0.4080
Epoch 4/100
Epoch 00003: val_loss improved from 1.94039 to 1.89781, saving model to snnweights_augmentation.best.hdf5
19s - loss: 1.9958 - acc: 0.3389 - val_loss: 1.8978 - val_acc: 0.3587
Epoch 5/100
Epoch 00004: val_loss improved from 1.89781 to 1.71767, saving model to snnweights_augmentation.best.hdf5
19s - loss: 1.8733 - acc: 0.3767 - val_loss: 1.7177 - val_acc: 0.4480
Epoch 6/100
Epoch 00005: val_loss improved from 1.71767 to 1.680

In [37]:
snn.load_weights('snnweights_augmentation.best.hdf5')

In [17]:
accuracy_test=snn.evaluate(test_tensors,test_targets)

950/950 [==============================] - 1s     


In [18]:
accuracy_test[1]*100

87.263157982575265

In [77]:
eval_paths=load_files('test/')
eval_filenames=eval_paths['filenames']

In [78]:
eval_tensors=paths_to_tensor(eval_filenames)
eval_tensors=eval_tensors/float(255)

In [79]:
eval_output=snn.predict(eval_tensors)
eval_output=[np.argmax(each) for each in eval_output]
eval_output=[target_names[each] for each in eval_output]

In [80]:
output=[(filename[10:],prediction) for filename,prediction in list(zip(eval_filenames,eval_output))]

In [81]:
output=[('file','species')] + output

In [82]:
import csv

 
with open('results1.csv', 'w',newline='')  as myfile:
    writer = csv.writer(myfile)
#     writer.writerow('file,species')
    writer.writerows(output)